In [33]:
from __future__ import print_function
import os
import neat
import json

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix


from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [34]:

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")
cuda_device = torch.device("cuda:1")

In [35]:
device

device(type='cpu')

# Iris Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [4]:
my_random_seed = 42
random.seed(my_random_seed)

In [5]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [6]:
iris = datasets.load_iris()
xs_raw = iris.data[:, :]
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = iris.target
ys_onehot = one_hot_encode(ys)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=45, random_state=42)

In [49]:
scaler.inverse_transform([0.,0.,0.,0.])

array([5.84333333, 3.05733333, 3.758     , 1.19933333])

In [50]:
scaler.inverse_transform([1.,1.,1.,1.])

array([6.66863463, 3.4917443 , 5.51740407, 1.95902596])

In [51]:
scaler.mean_

array([5.84333333, 3.05733333, 3.758     , 1.19933333])

In [54]:
scaler.scale_

array([0.82530129, 0.43441097, 1.75940407, 0.75969263])

In [58]:
scaler.transform([[0., 0., 0., 3.]])

array([[-7.08024256, -7.03788247, -2.1359505 ,  2.37025687]])

In [69]:
xs_raw.transpose()[2].mean()

3.7580000000000005

In [8]:
# # xs = torch.from_numpy(xs).to(device)
# # ys = torch.from_numpy(ys).to(device)
# X_train = torch.from_numpy(X_train).to(device)
# X_test = torch.from_numpy(X_test).to(device)
# y_train = torch.from_numpy(y_train).to(device)
# y_test = torch.from_numpy(y_test).to(device)

Let's have a look at the data we are working with

In [9]:
xs[:5]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ]])

In [10]:
ys[:5]

array([0, 0, 0, 0, 0])

In [11]:
ys_onehot[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

In [12]:
def eval_genomes(genomes, config):
    loss = nn.CrossEntropyLoss()
    loss = loss.to(device)

    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in X_train:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(y_train)))

## The competition

In [13]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);


In [14]:
# Use the forest's predict method on the test data
rf_preds = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(rf_preds - y_test)

In [15]:
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 0.01


In [16]:
rf_preds

array([1.001, 0.   , 1.995, 1.002, 1.12 , 0.   , 1.   , 1.969, 1.092,
       1.   , 1.955, 0.   , 0.   , 0.   , 0.   , 1.053, 2.   , 1.   ,
       1.001, 2.   , 0.   , 1.921, 0.   , 2.   , 2.   , 1.991, 1.986,
       2.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 1.96 ,
       1.003, 0.   , 0.   , 0.   , 1.983, 1.046, 1.002, 0.   , 0.   ])

In [17]:
errors = abs(rf_preds - y_test)

In [18]:
abs(rf_preds.round(0) - y_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
# Accuracy: 93.99 %.

Accuracy: nan %.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [20]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,rf_preds.round(0)))

[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]


# SVM

In [21]:
from sklearn.svm import SVC
svm_model=SVC()


In [22]:
svm_model.fit(X_train, y_train)


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [23]:
svm_preds=svm_model.predict(X_test)


In [24]:
print(confusion_matrix(y_train, svm_model.predict(X_train).round(0)))

[[31  0  0]
 [ 0 35  2]
 [ 0  2 35]]


In [25]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,svm_preds))

[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]


## Regression

In [26]:
from sklearn.linear_model import LinearRegression
regression_model=LinearRegression()

In [27]:
regression_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [28]:
pred=regression_model.predict(X_test)

In [29]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,pred.round(0)))

[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]


In [30]:
regression_model.coef_

array([-0.08874883, -0.025391  ,  0.46581225,  0.40210087])

In [31]:
print(confusion_matrix(y_train, regression_model.predict(X_train).round(0)))

[[31  0  0]
 [ 0 34  3]
 [ 0  1 36]]


# NNs

In [38]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_width=64):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_width) 
        self.fc2 = nn.Linear(hidden_width, hidden_width)
        self.fc3 = nn.Linear(hidden_width, hidden_width)
        self.fc4 = nn.Linear(hidden_width, hidden_width)
        self.fc5 = nn.Linear(hidden_width, output_size)  
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.relu(out)
        out = self.fc5(out)
        return out
    
class TabularDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, xs, ys):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.xs = xs
        self.ys = ys

    def __len__(self):
        return len(self.xs)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return (x, y)
    


In [39]:
batch_size = 1000
learning_rate = 0.0005
num_epochs = 200

In [41]:
train_data = TabularDataset(X_train, y_train)
train_loader = DataLoader(train_data, 
                           batch_size=batch_size, 
                           shuffle=True)

validate_data = TabularDataset(X_test, y_test)
validate_loader = DataLoader(dataset = validate_data,
                             batch_size=batch_size, 
                             shuffle=False)

total_step = len(train_loader)

my_random_seed = 42
random.seed(my_random_seed)
nn_model = NeuralNet(4, 3).to(cuda_device)

criterion = nn.CrossEntropyLoss().to(cuda_device)
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)  
for epoch in range(num_epochs):
    for i, (xsnn, ysnn) in enumerate(train_loader):  
        # Move tensors to the configured device
        xsnn = xsnn.float().to(cuda_device)
        ysnn = ysnn.view(-1, 1).float().to(cuda_device)

        # Forward pass
        outputs = nn_model(xsnn)
        train_loss = criterion(outputs, ysnn)
        

        # Backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        if (epoch+1) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, train_loss.item()))


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'

In [43]:
g

NameError: name 'g' is not defined

In [74]:
iris_df = pd.DataFrame(xs_raw, columns=iris.feature_names)
iris_df['label'] = iris.target

In [79]:
iris_df.groupby('label').mean()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
label,,,,
0,5.006,3.428,1.462,0.246
1,5.936,2.770,4.260,1.326
2,6.588,2.974,5.552,2.026


In [83]:
scaler.scale_

array([0.82530129, 0.43441097, 1.75940407, 0.75969263])

In [84]:
scaler.mean_

array([5.84333333, 3.05733333, 3.758     , 1.19933333])